<h1>Signal processing (in 1D)<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Signals:-descriptions-of-change-over-time-(or-space,-or-both)" data-toc-modified-id="Signals:-descriptions-of-change-over-time-(or-space,-or-both)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Signals: descriptions of change over time (or space, or both)</a></span></li><li><span><a href="#Frequency-domain-description-of-a-signal:-Fourier-transform" data-toc-modified-id="Frequency-domain-description-of-a-signal:-Fourier-transform-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Frequency-domain description of a signal: Fourier transform</a></span></li><li><span><a href="#Signal-resolution:-important-limitations-on-digital-signal-processing" data-toc-modified-id="Signal-resolution:-important-limitations-on-digital-signal-processing-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Signal resolution: important limitations on digital signal processing</a></span></li><li><span><a href="#Compound-waveforms" data-toc-modified-id="Compound-waveforms-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Compound waveforms</a></span></li><li><span><a href="#Manipulating-signals-in-the-frequency-domain" data-toc-modified-id="Manipulating-signals-in-the-frequency-domain-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Manipulating signals in the frequency domain</a></span></li></ul></div>

In [ ]:
import numpy as np
from numpy import *
np.set_printoptions(precision=2, suppress=True)
import time
from scipy.fft import fft, fftfreq, fftshift, ifft, ifftshift
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import clear_output, display, HTML
import ipywidgets as widgets

%matplotlib inline

In [ ]:
plt.style.use('seaborn-whitegrid')
plt.style.use('seaborn-poster')

## Signals: descriptions of change over time (or space, or both)

In [ ]:
duration = 3 # seconds
sampling_rate = 32 # sample per sec

times = linspace(start=0, stop=duration, num=sampling_rate*duration, endpoint=False)

In [ ]:
def sinusoid(frequency, amplitude=1, phase=0, mean=0, length=duration, sampling_rate=sampling_rate):
    xs = np.linspace(start=0, stop=length, num=int(sampling_rate*length), endpoint=False)
    phase = np.deg2rad(phase)
    return amplitude * np.cos(2*np.pi*frequency*xs + phase) + mean

In [ ]:
# Produce sinusoidal luminance (L) signal
original_frequency = 1
original_amplitude = 1
original_phase = 90
L = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase, length=duration, sampling_rate=sampling_rate)

### Changing luminance/intensity over time

In [ ]:
def plot_pixel(ax, value=0):
    """Plot 1 'pixel' on a background"""
    arr = np.zeros((3,3))
    arr[1,1] = value
    img = ax.imshow(arr, cmap="gray", vmin=-1, vmax=1, animated=True)
    ax.axis('off')
    return img

def anim_pixel(frame_num, img, waveform):
    """Update intensity of pixel"""
    img_array = img.get_array()
    img_array[1,1] = waveform[frame_num-1]
    img.set_array(img_array)
    return (img, )

In [ ]:
fig, pixel = plt.subplots(figsize=(6,5))
img = plot_pixel(pixel, 1)

plt.close('all')
display(HTML(FuncAnimation(fig, anim_pixel, fargs=(img, L), blit=True, frames=len(L), interval=1000/sampling_rate).to_jshtml()))
plt.close()

- 1 "pixel", changing in luminance
- some smooth oscillation

In [ ]:
L = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase, length=duration, sampling_rate=sampling_rate)

rng = np.random.default_rng(1234)
noise = rng.normal(0,2,times.shape)
L_in_noise = L + noise

L_in_noise = (2*((L_in_noise - L_in_noise.min()) / (L_in_noise.max() - L_in_noise.min())))-1

In [ ]:
fig, pixel = plt.subplots(figsize=(6,5))
img = plot_pixel(pixel, 1)

plt.close('all')
display(HTML(FuncAnimation(fig, anim_pixel, fargs=(img, L_in_noise), blit=True, frames=len(L_in_noise), interval=1000/sampling_rate).to_jshtml()))
plt.close()

- 1 "pixel", changing in luminance
- some more erratic change

### Expressing change in luminance (contrast)

In [ ]:
def plot_bar(ax, value=0):
    """Plot a single bar graph"""
    bars = ax.bar(x=0, height=value)
    ax.set_xlim(-1,1)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(True)
    ax.set_ylim(-original_amplitude*1.1,original_amplitude*1.1)
    ax.yaxis.tick_right()
    ax.set_ylabel("Contrast")
    ax.yaxis.set_label_position("right")
    return bars[0]

def anim_bar(frame_num, bar, waveform):
    """Update height of bar"""
    bar.set_height(waveform[frame_num-1])
    return (bar,)

In [ ]:
def anim_pix_bar(frame_num, pix, bar, waveform):
    """Update both intensity of pixel and height of bar"""
    anim_bar(frame_num, bar, waveform)
    anim_pixel(frame_num, pix, waveform)
    return (bar, pix)

In [ ]:
plt.close('all')
fig, (pix, b) = plt.subplots(1,2, figsize=(12,5))

pix = plot_pixel(pix)
bar = plot_bar(b)

L = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase)

display(HTML(FuncAnimation(fig, anim_pix_bar, fargs=(pix, bar, L), blit=True, frames=len(L), interval=1000/sampling_rate).to_jshtml()))
plt.close()

- 1  "pixel", changing in luminance
- Expressed as contrast between  [−1,1]
- Oscillating back and forth

### Tracing contrast over time

In [ ]:
def plot_waveform(ax, waveform, times, plot_frames=False, plot_wave=True, linestyle="-"):
    if not plot_wave:
        linestyle = " "
    marker = "o" if plot_frames else " "
    wav, = ax.plot(times, waveform, color='C0', marker=marker, linestyle=linestyle)

    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Contrast")

    return wav

def anim_waveform(frame_num, wav, times, waveform):
    wav.set_data(times[:frame_num], waveform[:frame_num])
    return wav

In [ ]:
def plot_trace(ax, t=0, val=0):
    dot, = ax.plot(t, val, 'C0o ')
    hline = ax.axhline(y=val, animated=True, linestyle="--", color="C0")
    return dot, hline
    
def anim_trace(frame_num, dot, hline, times, waveform):
    """Update trace"""
    dot.set_data(times[frame_num-1], waveform[frame_num-1])
    hline.set_data([0,2],[waveform[frame_num-1],waveform[frame_num-1]])
    return (dot, hline)

In [ ]:
def anim_pix_bar_trace(frame_num, pix, bar, dot, hline, times, waveform):
    anim_bar(frame_num, bar, waveform)
    anim_pixel(frame_num, pix, waveform)
    anim_trace(frame_num, dot, hline, times, waveform)
    return (pix, bar, dot, hline)

def anim_pix_bar_trace_wav(frame_num, pix, bar, dot, hline, wav, times, waveform):
    anim_bar(frame_num, bar, waveform)
    anim_pixel(frame_num, pix, waveform)
    anim_trace(frame_num, dot, hline, times, waveform)
    anim_waveform(frame_num, wav, times, waveform)
    return (pix, bar, dot, hline, wav)

In [ ]:
plt.close('all')
fig, (pix, b, w) = plt.subplots(1,3, figsize=(18,5))

pix = plot_pixel(pix)
bar = plot_bar(b)

b.yaxis.tick_right()
b.set_ylabel(" ")

L = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase)

wav = plot_waveform(w, L, times, plot_frames=False, plot_wave=False)
w.yaxis.set_label_position("right")
w.yaxis.set_ticklabels([])
w.set_ylim(-original_amplitude*1.1,original_amplitude*1.1)


dot, trace = plot_trace(w)

display(HTML(FuncAnimation(fig, anim_pix_bar_trace, fargs=(pix, bar, dot, trace, times, L), blit=True, frames=len(L), interval=1000/sampling_rate).to_jshtml()))
plt.close()

In [ ]:
plt.close('all')
fig, (pix, b, w) = plt.subplots(1,3, figsize=(18,5))

pix = plot_pixel(pix)
bar = plot_bar(b)

b.yaxis.tick_right()
b.set_ylabel(" ")

L = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase)

wav = plot_waveform(w, L, times, plot_frames=False, plot_wave=True)
w.yaxis.set_label_position("right")
w.yaxis.set_ticklabels([])
w.set_ylim(-original_amplitude*1.1,original_amplitude*1.1)

dot, trace = plot_trace(w)

display(HTML(FuncAnimation(fig, anim_pix_bar_trace_wav, fargs=(pix, bar, dot, trace, wav, times, L), blit=True, frames=len(L), interval=1000/sampling_rate).to_jshtml()))
plt.close()

- $1$ "pixel", changing in luminance contrast, between $[-1, 1]$
- Oscilates _sinusoidally_ $3$ times ($3$ full _periods_)
- In $3$ seconds, thus $\frac{3\ \mathrm{periods}}{3s} = \frac{1}{s} = 1\ \mathrm{Hz} $ frequency of oscilation

In [ ]:
original_frequency = 1 # Hz
duration           = 3 # s

L = sinusoid(frequency=original_frequency, length=duration)

Since our signal is a _digital / discrete_ signal,
we only have a limited number of _samples_ (not at every timepoint $t$)

In [ ]:
plt.close('all')
fig, (pix, b, w) = plt.subplots(1,3, figsize=(18,5))

pix = plot_pixel(pix)
bar = plot_bar(b)

b.yaxis.tick_right()
b.set_ylabel(" ")

L = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase)

wav = plot_waveform(w, L, times, plot_frames=True, plot_wave=True)
w.yaxis.set_label_position("right")
w.yaxis.set_ticklabels([])
w.set_ylim(-original_amplitude*1.1,original_amplitude*1.1)

dot, trace = plot_trace(w)

display(HTML(FuncAnimation(fig, anim_pix_bar_trace_wav, fargs=(pix, bar, dot, trace, wav, times, L), blit=True, frames=len(L), interval=1000/sampling_rate).to_jshtml()))
plt.close()

- $1$ "pixel", sinusoidally changing in luminance contrast between $[-1, 1]$, with a frequency of $1\ \mathrm{Hz}$
- Over $3s$, with $96$ frames (samples) in total, giving a $\frac{96}{3} = 32$ frames per second (fps) _sampling rate_

In [ ]:
duration           = 3  # s
sampling_rate      = 32 # fps
original_frequency = 1  # Hz

In [ ]:
assert duration*sampling_rate == len(L)
print(f"Length of waveform: {duration} * {sampling_rate} = {len(L)}")

We can describe any signal with a such a waveform:

In [ ]:
plt.close('all')
fig, (pix, b, w) = plt.subplots(1,3, figsize=(18,5))

pix = plot_pixel(pix)
bar = plot_bar(b)

b.yaxis.tick_right()
b.set_ylabel(" ")

wav = plot_waveform(w, L, times, plot_frames=True, plot_wave=True)
w.yaxis.set_label_position("right")
w.yaxis.set_ticklabels([])
w.set_ylim(-original_amplitude*1.1,original_amplitude*1.1)

dot, trace = plot_trace(w)

display(HTML(FuncAnimation(fig, anim_pix_bar_trace_wav, fargs=(pix, bar, dot, trace, wav, times, L_in_noise), blit=True, frames=len(L), interval=1000/sampling_rate).to_jshtml()))
plt.close()

though we may not be able to (easily) describe it with a single frequency and amplitude

For now, we are going to explore the tools of digital signal analysis using single sinusoids like this one,
and see if we can use some of those tools on other, more complex, signals as well.

In [ ]:
plt.close('all')
fig, (pix, b, w) = plt.subplots(1,3, figsize=(18,5))

pix = plot_pixel(pix)
bar = plot_bar(b)

b.yaxis.tick_right()
b.set_ylabel(" ")

L = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase)


wav = plot_waveform(w, L, times, plot_frames=True, plot_wave=True)
w.yaxis.set_label_position("right")
w.yaxis.set_ticklabels([])
w.set_ylim(-original_amplitude*1.1,original_amplitude*1.1)

dot, trace = plot_trace(w)

display(HTML(FuncAnimation(fig, anim_pix_bar_trace_wav, fargs=(pix, bar, dot, trace, wav, times, L), blit=True, frames=len(L), interval=1000/sampling_rate).to_jshtml()))
plt.close()

### Parameters of sinusoidal signals
- **amplitude**: how much the signal maximally deviates from the baseline
- **frequency**: number of periods per unit-time
- **phase**: where in the cycle it starts
- **mean**: an offset, or the baseline value around which the signal oscilates.

We can change these parameters of this sinusoidal signal.

Using the widget here,
see how changing each parameter changes the waveform.

What do you expect this to look like?
Does it match your expectations?

In [ ]:
# TODO: GENERATE ANIMATIONS HERE
def sinusoid_params(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase, mean=0):
    # Generate sinusoid
    L = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase)
    s = sinusoid(frequency=frequency, amplitude=amplitude, phase=phase, mean=mean)

    # Plot
    plt.figure(figsize=(16, 4))
    plt.plot(times, L, "k:", label="original")
    plt.ylim(-2,2)
    plt.plot(times, s)
    plt.xlabel("Time (s)")

In [ ]:
widgets.interactive(sinusoid_params, frequency=(0, sampling_rate/2, 1/duration), amplitude=(0, 2, 0.1), phase=(0,360,30), mean=(-1,1,0.1))

You should observe that you can directly link the parameters of the sinusoidal signal waveform
to the flickering of the light:
- The amplitude determines the (maximum) _contrast_ at which the light flickers -- lower amplitude means lower contrast flicker, higher amplitude means higher contrast
- The frequency determines the speed at which it flickers -- lower frequency means it takes longer to flicker the whole contrast range, higher frequency means it flickers faster
- The phase determines the starting contrast
- The mean determines the background/pedestal that the light flickers on -- mean $=0$ means the light flickers around the background luminance; if the the mean $\neq0$, then the contrast is modulated around some pedestal luminance.

## Frequency-domain description of a signal: Fourier transform
When we have a signal like this we can describe it as having some frequency, some amplitude, and some phase.
To describe our signal waveform in this way, we can apply the _Fourier transform_,
which analyzes a signal and describes it as one or more _frequency components_.

In [ ]:
L = sinusoid(frequency=original_frequency, phase=original_phase, amplitude=original_amplitude, length=duration, sampling_rate=sampling_rate)

Most (scientific) programming languages implement the _Fast Fourier Transform (FFT)_ algorithm to do this:

In [ ]:
FL = fft(L)
FL = FL.round()

The output of the Fourier transform is an array of complex numbers, of the same length as our original waveform.

In [ ]:
print(FL)

In [ ]:
print(f"Fourier-transform FL of our signal L is of type {FL.dtype}")
print(f"{len(L)} samples in our original signal")
print(f"{len(FL)} components in the Fourier transform of our signal")

assert L.shape == FL.shape

Each entry in this Fourier transform corresponds to a component with a specific frequency.
For each of these frequency components,
the complex number encodes the amplitude and phase
with which it appears in our signal.

### Complex numbers encode two (separate) pieces of information
A complex number can be consider a point (vector) in the 2D complex plane:

In [ ]:
complex_number = 2 + 3j

In [ ]:
fig = plt.figure(figsize=(5,5))
plt.plot([0, complex_number.real], [0, complex_number.imag], marker="<")
plt.plot([0, complex_number.real],[complex_number.imag]*2, color="C0", linestyle=":")
plt.plot([complex_number.real]*2,[0,complex_number.imag], color="C0", linestyle=":")

plt.xlabel("Real part")
plt.ylabel("Imaginary part (j)")
plt.xlim(-1, 4)
plt.ylim(-1, 4)
plt.axhline(y=0, color="k")
plt.axvline(x=0, color="k")

plt.show()

We can derive two aspects of the frequency component from such a complex number:
 - its amplitude, as the magnitude (absolute value) of the complex number
 $$amplitude = |z| = \sqrt{real^2  + imaginary^2}$$
 - its phase, as the angle of the complex number

### Describing frequency components

Let's look just at the amplitudes:
We'll take our Fourier transform, and convert it to an amplitude spectrum.
Since the amplitude of each frequency component is
encoded as the magnitude of the complex number,
we can get the amplitude spectrum as the absolute values
of the complex entries

In [ ]:
amplitude_spectrum = abs(FL)

In [ ]:
print(amplitude_spectrum)

In [ ]:
plt.figure()
plt.stem(np.abs(FL))
plt.xlabel("Frequency component #")
plt.ylabel("Amplitude")
plt.show()

We see two peaks, which means two _frequency components_
have a high(est) amplitude.
Let's find out what the first of these is:

In [ ]:
peak_frequency_idx = argmax(amplitude_spectrum)

In [ ]:
print(f"Maximum amplitude is for frequency component with index {peak_frequency_idx}")

#### Frequency information
To figure out what frequency this component has
we can use the `fftfreq()` function,
which gives a vector of the frequencies extracted
for given a number of samples and the sampling interval
(sampling interval, i.e., the interval between samples,
 can be calculated as unit-time divided by sampling rate,
 i.e., the inverse of sampling rate)

In [ ]:
freqs = fftfreq(len(L), d=(1/sampling_rate))

In [ ]:
print(freqs)

These frequencies are expressed in cycles per unit
-- per unit-time in our case, i.e. cycles-per-second (fps), or Hz.

In [ ]:
peak_frequency = freqs[argmax(amplitude_spectrum)]

In [ ]:
print(f"The maximum amplitude in our signal is for frequency {peak_frequency:.1f} Hz")

assert peak_frequency == original_frequency

In [ ]:
plt.figure()
plt.stem(freqs, amplitude_spectrum)
plt.ylabel("Amplitude")
plt.xlabel("Frequency (Hz)")
plt.show()

- Fourier transform describes both positive and _negative_ frequencies
- For real signals, these will always be mirror symmetric around the origin ($0$)
- Thus, we also have a peak in amplitude spectrum at negative frequency component

We can also see now why we have two peaks in the amplitude spectrum.
The Fourier transform extracts both positive and _negative_ frequencies.
While in the real world negative frequencies don't make a lot of sense,
mathematically they're just as valid as positive frequencies.
Thus, the Fourier transform needs to return both of these
to give a full mathematical transformation of the signal into the frequency domain.

In our cases, the positive and negative frequencies
will always be mirror symmetric around the origin ($0$).
We can see that our other peak is at the negative of the frequency that we put in.

You could plot just the positive frequencies,
but convention is mostly to plot both.

##### FFT shift
We can also see that in the output of the FFT, the positive frequencies are listed first, followed by the negative frequencies. When we plot them, that does not matter much, because the plotting function automatically sorts the axis in this case. However, it's nicer to rearrange this output list of frequencies, for which we can use the `fftshift()` function -- which we then also have to do for our spectrum.

In [ ]:
freqs = fftshift(freqs)
FL = fftshift(FL)
amplitude_spectrum = fftshift(amplitude_spectrum)

In [ ]:
print(fftfreq(len(L), d=(1/sampling_rate)))

In [ ]:
print(freqs)

#### Amplitude
Let's look at the amplitude of our signal.
Since we're looking at an amplitude spectrum,
the height (y-value) of our peak should be the amplitude of our original signal, right?

In [ ]:
print(f"Recovered amplitude = {np.max(amplitude_spectrum):.1f} (original = {original_amplitude:.1f})")

This doesn't match. Instead, the recovered amplitude is half the number of samples times higher...

In [ ]:
print(f"{duration}s * {sampling_rate}fps = {len(L)} frames (= samples)")

The amplitude of each frequency component in the Fourier transform,
is the *total amplitude* over the whole signal,
i.e. the integrated (summed) amplitude over the number of samples in the signal.

The amplitude that we defined earlier, is the *peak amplitude*,
i.e., the maximum deviation from 0 that the signal will take each period.

For sinusoidal signals, the integrated amplitude,
is the peak amplitude times the total number of samples.
Conversely, for the sinusoidal frequency components represented in an amplitude spectrum,
we can get the peak amplitude by dividing the amplitude spectrum by the number of samples:

Convention is thus to divide amplitude spectrum by the total number of samples

In [ ]:
peak_amplitudes = amplitude_spectrum / (duration*sampling_rate)

In [ ]:
print(peak_amplitudes)

But wait, now the peak amplitude seems to be half our original amplitude? Remember that we have both the positive *and* the negative frequency components: what we think of as a single signal with a positive frequency and some amplitude, the Fourier transform considers to be a combination of a positive and negative frequency component, which then each "get" half the peak amplitude.

In [ ]:
plt.figure()
plt.stem(freqs, peak_amplitudes)
plt.ylabel("Peak amplitude")
plt.xlabel("Frequency (Hz)")
plt.show()

In [ ]:
assert peak_amplitudes.sum() == original_amplitude

This makes sense now:
we see the peak amplitude with which we created our original signal,
at the frequency of our sinusoid
(and at its negative complement)

#### Exploring amplitude spectra

Use the widget to change the frequency and amplitude
of the signal.

How do you expect the amplitude spectrum to change?
Does it match your expectations?

In [ ]:
def plot_sinusoids(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase, mean=0):
    # Generate sinusoid
    s = sinusoid(frequency=frequency, amplitude=amplitude, phase=phase, mean=mean)
    L = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase)
    

    # Transform
    freqs = fftshift(fftfreq(len(times), d=(1/sampling_rate)))
    Fs = fftshift(fft(s)).round()
    amplitude_spectrum = np.abs(Fs)
    peak_amplitudes = amplitude_spectrum / (duration*sampling_rate)
    
    # Plot
    fig, (ax_wav, ax_amp) = plt.subplots(1,2, figsize=(16, 5))
    ax_wav.plot(times, L, "k:", label="original")
    ax_wav.set_ylim(-2,2)
    ax_wav.plot(times, s)
    ax_wav.set_xlabel("Time (s)")
    
    ax_amp.stem(freqs, peak_amplitudes)
    ax_amp.set_ylim(0,2)
    ax_amp.axvline(original_frequency, linestyle=":", color="k")
    ax_amp.axvline(-original_frequency, linestyle=":", color="k")
    ax_amp.axhline(original_amplitude/2, linestyle=":", color="k")
    ax_amp.set_ylabel("Peak amplitude")
    ax_amp.yaxis.tick_right()
    ax_amp.yaxis.set_label_position("right")
    ax_amp.set_xlabel("Frequency (Hz)")

interactive_plot = widgets.interactive(plot_sinusoids, frequency=(0, sampling_rate/2, sampling_rate/len(L)), amplitude=(0, 2, 0.1), phase=(0,360,30), mean=(-1,1,0.1))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

- Frequency of the input signal changes where the peak in the amplitude spectrum is. Increasing the frequency moves the peak to the right (and the negative frequency peak to the left, i.e., pushes them apart).
- Amplitude of the singal changes the height of the peak.
- Phase does not change the amplitude spectrum.

#### 0 Hz frequency
The component at 0 Hz is often called the _direct current_ or _DC_ component.
It's the remaining "signal" when there is no oscillation;
it's the total integrated sum of the signal over the whole period;
for a symmetric signal like a sinusoid, it is thus also the mean.

Remember: in our example of a flashing light,
the mean of that signal (the flashing) is the background/pedestal on which the flashing occurs.
If we set our mean $=0$,
then the contrast is modulated around the background luminance;
if the the mean $\neq0$, then the contrast is modulated around some pedestal luminance.

#### Power spectrum
The _power spectrum_ is defined as the square of the amplitude spectrum:
$$ power = amplitude^2 $$

In [ ]:
power_spectrum = peak_amplitudes ** 2

In [ ]:
plt.figure()
plt.stem(freqs, power_spectrum)
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.show()

### Phase
Phase information does not show up in the amplitude spectrum.
As mentioned above, the complex-valued output of the Fourier transform
captures the amplitude as the magnitude of the complex number,
and the phase as the _angle_ of the complex number.

In [ ]:
L = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase)
FL = fftshift(fft(L).round())
amplitudes = abs(FL)
FL.imag[FL.imag==-0] = 0
FL.real[FL.real==-0] = 0

In [ ]:
phases = rad2deg(angle(FL))

In [ ]:
phases = phases
print(phases)

In [ ]:
print(f"Peak frequency component has phase {abs(phases[np.argmax(amplitudes)]):.2f} degrees (original = {original_phase:.2f})")

#### Phase spectrum

In [ ]:
def plot_sinusoids(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase, mean=0):
    # Generate sinusoid
    L = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase, mean=0)
    s = sinusoid(frequency=frequency, amplitude=amplitude, phase=phase, mean=mean)
    
    # Transform
    freqs = fftshift(fftfreq(len(times), d=(1/sampling_rate)))
    Fs = fftshift(fft(s)).round()
    Fs.imag[Fs.imag==-0] = 0
    Fs.real[Fs.real==-0] = 0
    phases = np.rad2deg(np.angle(Fs.round()))
    amplitude_spectrum = np.abs(Fs)
    
    # Plot
    fig, (ax_wav, ax_phase) = plt.subplots(1,2, figsize=(16, 5))
    ax_wav.plot(times, L, "k:", label="original")
    ax_wav.set_ylim(-2,2)
    ax_wav.plot(times, s)
    ax_wav.set_xlabel("Time (s)")
    
    ax_phase.stem(freqs, phases)
    ax_phase.set_ylim(-180,180)
    ax_phase.set_ylabel("Phase (degrees)")
    ax_phase.yaxis.tick_right()
    ax_phase.yaxis.set_label_position("right")
    ax_phase.set_xlabel("Frequency (Hz)")
    
interactive_plot = widgets.interactive(plot_sinusoids, frequency=(0, sampling_rate/2, sampling_rate/len(L)), amplitude=(0, 2, 0.1), phase=(0,360, 30), mean=(-1,1,0.1))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

- DC component: if positive gets $phase = 0$; if negative: $phase = (-)180$

## Signal resolution: important limitations on digital signal processing
Two aspects of our signal and its Fourier transform that we haven't addressed so far,
are
- the sampling rate: the number of samples or frames per second)
- the length: the number of samples in total, or the duration

In [ ]:
def plot_sinusoids(sampling_rate=sampling_rate, duration=duration):
    # Generate timebase
    times = np.linspace(start=0, stop=duration, num=int(sampling_rate*duration), endpoint=False)

    # Generate sinusoid
    s = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase, mean=0, length=duration, sampling_rate=sampling_rate)

    
    # Plot
    fig, ax_wav = plt.subplots(1,1, figsize=(16, 5))
    ax_wav.set_ylim(-2,2)
    ax_wav.plot(times, s, "o:")
    ax_wav.set_xlabel("Time (s)")

plt.close('all')
interactive_plot = widgets.interactive(plot_sinusoids, sampling_rate=(1,64), duration=(0.1,5,1/sampling_rate))
#output = interactive_plot.children[-1]
#output.layout.height = '350px'
display(interactive_plot)
plt.close()

What you should observe,
is that changing the sampling rate
doesn't necessarily change any of the signal parameters
(frequency, amplitude, phase).
It does, however, change the smoothness, or precision, of the signal;
the lower the sampling rate, the sharper/blockier the signal is.

Similarly, changing the duration does not affect the signal parameters.
Instead, it determines how much of our signal we see.
Specifically, note that the number of full periods
(where the signal has returned to its starting position)
only happens when the $\mathrm{duration} \times \mathrm{frequency}$ is an integer.

- $\mathrm{number\ of\ samples}$ changes with both $\mathrm{sampling\ rate}$ and $\mathrm{duration}$
- output of DFT is same length as input signal
- i.e., $\mathrm{number\ of\ frequency\ components} = \mathrm{number\ of\ samples}$

What does change with both the $\mathrm{sampling\ rate}$ and the $\mathrm{duration}$
is the total $\mathrm{number\ of\ samples}$.
This is relevant, because the output of the FFT is the same length as the input.
In other words, the Discrete Fourier Transform (DFT) describes the signal in a number of $\mathrm{frequency\ components}$ in the frequency domain
that is equal to the $\mathrm{number\ of\ samples}$ in the original (time) domain.

In [ ]:
assert len(FL) == len(L)

Thus, which frequency components we can identify in our signal,
depends on the number of samples.

Let's take a look at the frequencies in our original Fourier transform:

In [ ]:
print(freqs)

- range between $ \pm \frac{sampling\ rate}{2} \mathrm{Hz}$
- ${\#\ samples} = {\#\ frequency\ components}$
- so whole range of ${sampling\ rate}$ gets divided into ${\#\ samples}$
- giving steps of $ \frac{sampling\ rate}{\#\ samples} = \frac{sampling\ rate}{sampling\ rate\ \times\ duration} = \frac{1}{duration}\ \mathrm{Hz}$
- smallest nonzero frequency is a single step: $ \pm \frac{sampling\ rate}{\# \ samples} = \pm \frac{1}{duration}$

In [ ]:
assert np.allclose(np.linspace(-(sampling_rate/2), sampling_rate/2-1/duration, len(L)), freqs)

In [ ]:
def frequencies(sampling_rate, duration):
    n_samples = int(duration * sampling_rate)
    max_freq = sampling_rate/2
    smallest_freq = sampling_rate / n_samples
    print(f"{n_samples} freq. components, between +-[{smallest_freq:.2f},{max_freq}] Hz, in steps of {smallest_freq:.2f} Hz")
    
w = widgets.interactive(frequencies, sampling_rate=(1,64,1), duration=(0,5,0.1))
display(w)

#### Maximum frequency, and beyond
- Nyquist limit = highest frequency component = $\frac{sampling\ rate}{2}$
- Limitation of discrete signals (not just DFT)

As you can see, the highest frequency component that the DFT can consider, is $\frac{sampling\ rate}{2}$, which is called the Nyquist frequency. This is not just a limitation of the DFT, but a limitation of discrete signals with a finite sampling rate in general. As the frequency increases, there are fewer and fewer samples per period of the signal. As a result, each period will look blockier, which can lead to some ugly artefacts.

Use this widget to play around with signal frequency, signal frequency, and phase

Note specifically what happens when the frequency of the signal is the Nyquist limit, or above.

In [ ]:
def plot_two(sampling_rate = sampling_rate, frequency=3, phase=0):
    duration = 1
    nyquist = sampling_rate / 2
    alias_freq = nyquist-(frequency-nyquist) if frequency > nyquist else frequency
    
    # Generate sinusoids
    actual = sinusoid(length=duration, sampling_rate=10*sampling_rate, frequency=frequency, phase=phase)
    alias = sinusoid(length=duration, sampling_rate=sampling_rate, frequency=alias_freq, phase=phase)
    
    # Generate corresponding timebases
    times_low = np.linspace(start=0, stop=duration, num=int(sampling_rate*duration), endpoint=False)
    times_high = np.linspace(start=0, stop=duration, num=int(10*sampling_rate*duration), endpoint=False)
    
    # Plot
    print(f"Nyquist limit = {sampling_rate/2}")
    plt.figure(figsize=(16,5))
    plt.plot(times_high, actual, "C1:", label=f"{frequency:.2f} Hz")
    plt.plot(times_low, alias, "C0o-", label=f"{alias_freq:.2f} Hz")
    plt.legend()
    plt.ylim(-2,2)
    plt.xlabel("Time (s)")

# Setup widgets, that can depend on each other
sr_slider = widgets.IntSlider(min=1, value=32, max=64)
freq_slider = widgets.FloatSlider(min=0, value=2, max=sr_slider.value, step=1)
def update_f(*args):
    freq_slider.max = sr_slider.value
sr_slider.observe(update_f)

# Link widgets to function
widgets.interactive(plot_two, sampling_rate=sr_slider, frequency=freq_slider, phase=(0,360,15))

- _Aliasing_: frequency $\mathrm{Nyquist}+N$ looks like $\mathrm{Nyquist}-N$

This effect is called _aliasing_: a frequency beyond the resolution (sampling rate) of our analysis window, gets _aliased_ as a lower frequency. Since the waveform of the too-high frequency is identical to that of the waveform of a frequency it gets aliased as, its Fourier transform will necessarily also be identical.

#### Falling between the frequency gaps

- ${\#\ frequency\ components} = {\#\ samples}$
- so whole range of ${sampling\ rate}$ gets divided into ${\#\ samples}$
- giving steps of $ \frac{sampling\ rate}{\#\ samples} = \frac{sampling\ rate}{sampling\ rate\ \times\ duration} = \frac{1}{duration}\ \mathrm{Hz}$
- smallest nonzero frequency is a single step: $ \pm \frac{sampling\ rate}{\# \ samples} = \pm \frac{1}{duration}$

In [ ]:
print(freqs)

We also saw that the set of frequencies that the DFT analyzes
is determined by the total number of samples.
Specifically, the number of frequency components (in the frequency domain)
is equal to the number of samples (in the original (time) domain).
The frequency components are equally spaced,
such that they are $\frac{sampling\ rate}{\#\ samples} = \frac{1}{duration}$ Hz apart.

This is also why we need to put those pieces of information into `fftfreq`:
```
freqs = fftfreq(len(n_samples), d=(1/sampling_rate))
```

Another way to think of this is that
each frequency analyzed by the DFT is an integer multiple of the smallest possibe frequency,
which is also called the _fundamental frequency_

Intuitively: fundamental frequency as
the frequency that has one full period, i.e., makes exactly one full cycle
in the whole sample.
Thus, it is $1$ cycle per $duration$, i.e., $\frac{1}{duration}$
(does not depend on the sampling rate)

(https://www.princeton.edu/~cuff/ele201/kulkarni_text/frequency.pdf)

In [ ]:
fundamental_frequency = 1/duration

In [ ]:
print(f"Fundamental frequency = 1 / {duration:.2f}s = {fundamental_frequency:.2f} Hz")

In [ ]:
assert fftfreq(len(times), d=(1/sampling_rate))[1] == fundamental_frequency

In [ ]:
fundamental_sinusoid = sinusoid(frequency=fundamental_frequency, phase=90)
plt.figure(figsize=(5,5))
plt.plot(times, fundamental_sinusoid)
plt.xlabel('Time (s)')
plt.title(f"Sinusoid with fundamental frequency {fundamental_frequency:.2f} Hz, for duration {duration}s")
plt.show()

So the frequency components that the DFT will pick up on
are integer multiples of the smallest (fundamental) frequency.
But what if our signal has a frequency that isn't a nice integer multiple of the smallest frequency?

In [ ]:
frequency = 12.5 * fundamental_frequency

In [ ]:
# Generate sinusoid
s = sinusoid(frequency=frequency)

# Transform
freqs = fftshift(fftfreq(len(times), d=(1/sampling_rate)))
Fs = fftshift(fft(s)).round()
Fs.real[Fs.real==-0] = 0
Fs.imag[Fs.imag==-0] = 0
phases = np.rad2deg(np.angle(Fs))

amplitude_spectrum = np.abs(Fs)
peak_amplitudes = amplitude_spectrum / (duration*sampling_rate)


# Plot
fig, (ax_wav, ax_amp, ax_phase) = plt.subplots(1,3, figsize=(16, 6))
ax_wav.plot(times, s)
ax_wav.set_xlabel("Time (s)")
ax_wav.set_ylim(-2,2)
ax_wav.set_title("Waveform")

ax_amp.stem(freqs, peak_amplitudes)
ax_amp.set_ylim(0,0.8*original_amplitude)
ax_amp.set_ylabel("Peak amplitude")
ax_amp.yaxis.tick_right()
ax_amp.yaxis.set_label_position("right")
ax_amp.set_xlabel("Frequency (Hz)")
ax_amp.set_title("Amplitude spectrum")

ax_phase.stem(freqs, phases)
ax_phase.set_ylim(-180,180)
ax_phase.set_ylabel("Phase (degrees)")
ax_phase.yaxis.tick_right()
ax_phase.yaxis.set_label_position("right")
ax_phase.set_xlabel("Frequency (Hz)")
ax_phase.set_title("Phase spectrum")

plt.show()

What you should observe is that that the peaks in the amplitude spectrum
are no longer sharp, single points.
Instead, two "smeared out" peaks appear: on centered on our positive frequency
and a symmetric one centered on the negative version of our frequency.

Since the DFT cannot neatly assign the amplitude in the signal
to a single frequency component that its analyzing,
it has to "distribute" the amplitude over a number of components
mostly ones _near_ the signal frequency.

This kind of smearing will happen any time the signal
is not a perfect integer multiple of the fundamental frequency (i.e., $\frac{1}{\mathrm{duration}}$).
Since most realistic signals are not pure sinewaves anyway,
they will always show this kind of smearing.

**Resolution of DFT is limited by number of samples in original signal**
- Number of frequency components $=$ number of samples
- Nyquist limit: highest frequency $= \frac{sampling\ rate}{2}$
- Fundamental frequency: lowest (non-DC) frequency $= \frac{1}{duration}$
- All other frequency components are integer multiples of fundamental frequency.



- Higher sampling rate: higher frequencies can be analyzed
- Longer signal (more samples): more frequency components,
i.e., better resolution in the frequency domain
- Thus: longer in one domain is better resolution in other domain (Heisenberg/Fourier uncertainty principle [[1]])

[1]: https://en.wikipedia.org/wiki/Uncertainty_principle#Wave_mechanics_interpretation:~:text=Mathematically%2C%20in%20wave,the%20wavenumber.

## Compound waveforms

For single sinewaves the Fourier transform is probably a bit overkill to describe the signal.
Where it really shines, however, is for signals that are not neat single sinewaves
-- either because they are combinations of single sinewaves (think of chords of musical notes),
or because they're aperiodic signals.

The Fourier _theorem_:
any signal can be completely described as a weighted combination of sines and cosines.

This holds true for an infinite signal that is described with infinitely many sinewaves,
but even for our DFT version with finite samples (and thus finite frequency components)
it is a very useful property of signals.

We'll now look at what compound waveforms (combinations of signals)
look like, both in the time domain, and the frequency domain.

### Simple addition
The simplest combination of two sinewaves that we can do
is to add some sinusoid to itself.

In [ ]:
L_unit = sinusoid(original_frequency, amplitude=original_amplitude)
L_summed = L_unit + L_unit

In [ ]:
# Transform
freqs = fftshift(fftfreq(len(times), d=(1/sampling_rate)))
FL_unit = fftshift(fft(L_unit))
FL_summed = fftshift(fft(L_summed))
peak_amps_unit = np.abs(FL_unit) / (duration*sampling_rate)
peak_amps_summed = np.abs(FL_summed) / (duration*sampling_rate)

# Plot
plt.close()
fig, axs = plt.subplots(2,3, sharey='row', sharex="row", figsize=(16, 7))

axs[0, 0].plot(times, L_unit, label="unit")
axs[0, 0].set_title("Unit")
axs[0, 1].plot(times, L_summed, label="summed")
axs[0, 1].set_title("Summed")
axs[0, 2].set_visible(False)


axs[1,0].stem(freqs, peak_amps_unit, label="unit")
axs[1,1].stem(freqs, peak_amps_summed, label="summed")
axs[1, 2].set_visible(False)

plt.show()

Since at every point the two unit-waveforms are identical
it means that when summing them
at every point we get double the value of the original unit-waveform.

Thus, we end up with a waveform of the same frequency, and double the amplitude.

We can see that this addition works in the time-domain,
and that the amplitude spectrum reflects the summing as well.

Since multiplication is just repeatedly summing copies of the same waveform,
a sum of unit-waveforms should be the same as waveform with scaled amplitude:
$$ k \cdot s(t) = \sum^{k} s(t) = s_0(t) + s_1(t) + ... + s_k(t) $$

In [ ]:
L_summed = L_unit + L_unit + L_unit
L_scaled = 3*L_unit

In [ ]:
# Transform
freqs = fftshift(fftfreq(len(L_unit), d=(1/sampling_rate)))
FL_unit = fftshift(fft(L_unit))
FL_scaled = fftshift(fft(L_scaled))
FL_summed = fftshift(fft(L_summed))
peak_amps_unit = np.abs(FL_unit) / len(L_unit)
peak_amps_scaled = np.abs(FL_scaled) / len(L_unit)
peak_amps_summed = np.abs(FL_summed) / len(L_unit)

# Plot
plt.close()
fig, axs = plt.subplots(2,3, sharey='row', sharex="row", figsize=(16, 7))

axs[0, 0].plot(times, L_unit, label="unit")
axs[0, 0].set_title("Unit")
axs[0, 1].plot(times, L_summed, label="summed")
axs[0, 1].set_title("Summed")
axs[0, 2].plot(times, L_scaled, label="scaled")
axs[0, 2].set_title("Scaled")

axs[1,0].stem(freqs, peak_amps_unit, label="unit")
axs[1,1].stem(freqs, peak_amps_summed, label="summed")
axs[1,2].stem(freqs, peak_amps_scaled, label="scaled")


plt.show()

- Addition in the signal-domain == addition in the frequency domain

### Adding different sinusoids

Here we have our original sinusoid,
and you can select the parameters of a second sinusoid.
These two then get summed into a compound waveform,
and the amplitude spectra of this compoud waveform
and the two components are plot.

In [ ]:
def plot_compounds(frequency=1, amplitude=1, phase=0, mean=0):
    # Generate sinusoid
    L_1 = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase, length=duration, sampling_rate=sampling_rate, mean=0)
    L_2 = sinusoid(frequency=frequency, amplitude=amplitude, phase=phase, mean=mean, length=duration, sampling_rate=sampling_rate)
    L_summed = L_1 + L_2    
    
    # Transform
    freqs = fftshift(fftfreq(len(times), d=(1/sampling_rate)))
    FL_1 = fftshift(fft(L_1))
    FL_2 = fftshift(fft(L_2))
    FL_summed = fftshift(fft(L_summed))
    peak_amps_1 = np.abs(FL_1) / (2*sampling_rate)
    peak_amps_2 = np.abs(FL_2) / (2*sampling_rate)
    peak_amps_summed = np.abs(FL_summed) / (2*sampling_rate)
    
    # Plot
    plt.close()
    fig, axs = plt.subplots(2,3, sharey='row', sharex="row", figsize=(16, 8))
    axs[0, 0].plot(times, L_1, label=f"{original_frequency} Hz")
    axs[0, 0].set_title(f"{original_frequency} Hz")
    axs[0,1].plot(times, L_2, label=f"{frequency} Hz")
    axs[0, 1].set_title(f"{frequency} Hz")
    axs[0,2].plot(times, L_summed, label="summed")
    axs[0, 2].set_title("Summed")
    axs[0, 1].set_xlabel("Time (s)")
    axs[0, 0].set_ylim(-4,4)

    
    axs[1,0].stem(freqs, peak_amps_1, label=f"{original_frequency} Hz")
    axs[1,1].stem(freqs, peak_amps_2, label=f"{frequency} Hz")
    axs[1,2].stem(freqs, peak_amps_summed, label="summed")   
    axs[1,0].set_ylabel("Peak amplitude")
    axs[1,1].set_xlabel("Frequency (Hz)")
    axs[1,0].set_ylim(0,3)

    
interactive_plot = widgets.interactive(plot_compounds, frequency=(0, sampling_rate/2, 1/duration), amplitude=(0, 2, 0.1), phase=(0,180,30), mean=(-1,1,0.1))
output = interactive_plot.children[-1]
#output.layout.height = '350px'
interactive_plot

We can see that the combined waveform is a perfection composition of the two components.
The amplitude spectrum also reflects this:
it contains both the original frequency and its amplitude,
and the second component at its amplitude.

Hopefully you already observe (depending on the parameters you chose)
that compound waveforms can be surprisingly hard to identify.
Here there are just two sinusoidal components,
but the compound can already seem erratic. The Fourier transform can take this hard to identify pattern,
and because it breaks it down into constinuent components,
reveal the simpler underlying structure.

### Signal in "noise"
Since frequency analysis can reveal some underlying structure
in a signal that looks erratic,
it is especially useful for problems where there is a signal "hidden" in noise.

#### Additional components as noise

Here we've added a bunch of additional sinusoids, each with a random amplitude, and crucially, a random phase.
This means that all those sinusoids are not aligned, and the signal becomes very erratic in the time domain.
However, in the frequency domain, we can clearly see our original, low-frequency signal,
and a bunch of high frequency noise.

In [ ]:
rng = np.random.default_rng(1234)

hf_noise = np.zeros(L.shape)
for i in range(30):
    f = freqs[-i]
    hf_noise += sinusoid(frequency=f, amplitude=rng.uniform(0,.5,1), phase=rng.uniform(0,180,1))
L_in_hf_noise = L + hf_noise

Fy = fftshift(fft(L_in_hf_noise))
amplitude_spectrum = np.abs(Fy) / (duration*sampling_rate)
phase_spectrum = np.rad2deg(np.angle(Fy.round()))
phase_spectrum = phase_spectrum % 180 * np.sign(phase_spectrum)

fig, axs = plt.subplots(1,3, figsize=(16,5))
plt.subplot(1,3,1)
plt.title("Waveform")
plt.plot(times, L_in_hf_noise)
plt.xlabel('Time (s)')
plt.subplot(1,3,2)
plt.stem(freqs, amplitude_spectrum)
plt.xlabel("Frequency (Hz)")
plt.title("Amplitude spectrum")
plt.subplot(1,3,3)
plt.stem(freqs, phase_spectrum)
plt.xlabel("Frequency (Hz)")
plt.title("Phase spectrum")
plt.show()

**This is the goal of spectral analysis: by using an alternative representation of the signal, provide insights into its structure**

#### Random noise

Here, we add a pseudorandom noise-value (drawn from a normal distribution)
to each sample of our singal.
In the time domain, the waveform indeed looks random;
it's hard to identify the original signal.

However, in the amplitude spectrum we can clearly see
the peak frequency stand out.
This frequency-domain representation of our signal
allows us to identify the signal easily.


The spectrum also shows a key property of random noise:
it is composed of all frequency components, with various amplitudes
(it "contains power at all frequencies").
True "white noise",
where each sample is completely independent from all other samples
will have uniform power at all frequencies.

In [ ]:
rng = np.random.default_rng(1234)

w_noise = rng.normal(0,2,times.shape)
L_in_w_noise = L + w_noise

Fy = fftshift(fft(L_in_w_noise))
amplitude_spectrum = np.abs(Fy) / (duration*sampling_rate)
phase_spectrum = np.rad2deg(np.angle(Fy.round()))
phase_spectrum = phase_spectrum % 180 * np.sign(phase_spectrum)

fig, axs = plt.subplots(1,3, figsize=(16,5))
plt.subplot(1,3,1)
plt.title("Waveform")
plt.plot(times, L_in_w_noise)
plt.xlabel('Time (s)')
plt.subplot(1,3,2)
plt.stem(freqs, amplitude_spectrum)
plt.xlabel("Frequency (Hz)")
plt.title("Amplitude spectrum")
plt.subplot(1,3,3)
plt.stem(freqs, phase_spectrum)
plt.xlabel("Frequency (Hz)")
plt.title("Phase spectrum")
plt.show()

## Manipulating signals in the frequency domain

### Adjusting amplitude, phase, of single sinusoid

Since the Fourier transform of our signal,
and the amplitude and phase spectra are also just arrays of numbers,
we can manipulate them directly.

Let's manipulate a single sinusoid, in the frequency domain:

In [ ]:
# Original
L = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase)

# Fourier transform
FL = fftshift(fft(L)).round()
FL.real[FL.real == -0] = 0
FL.imag[FL.imag == -0] = 0

# Derive amplitude & phase spectra
freqs = fftshift(fftfreq(len(times), d=(1/sampling_rate)))
amplitude_spectrum = np.abs(FL) / (duration * sampling_rate)
phase_spectrum = np.rad2deg(np.angle(FL))
phase_spectrum = phase_spectrum

In [ ]:
# Plot
fig, axs = plt.subplots(1,3, figsize=(16, 4), sharey="col", sharex="col")

axs[0].set_title("Original waveform")
axs[0].plot(times, L)
axs[0].set_xlabel("Time (s)")

axs[1].set_title("Original amplitude spectrum")
axs[1].stem(freqs, amplitude_spectrum)

axs[2].set_title("Original phase spectrum")
axs[2].stem(freqs, phase_spectrum)

axs[1].set_title("Adjusted amplitude spectrum")
axs[1].set_xlabel("Frequency (Hz)")

axs[2].set_title("Adjusted phase spectrum")
axs[2].set_xlabel("Frequency (Hz)")

plt.show()

Here, we scale the amplitude spectrum
(i.e., the amplitude of the one sinusoid).
We'll keep the phase as it is, but you could also change that.

In [ ]:
amplitude_adjust = amplitude_spectrum / 2
phase_adjust = phase_spectrum

In [ ]:
# Plot
fig, axs = plt.subplots(1,3, figsize=(16, 4), sharey="col", sharex="col")

axs[0].set_title("Waveform")
axs[0].plot(times, L, visible=False)
axs[0].set_xlabel("Time (s)")

axs[2].set_title("Original phase spectrum")
axs[2].stem(freqs, phase_spectrum)
axs[2].set_xlabel("Frequency (Hz)")

axs[1].set_title("Adjusted amplitude spectrum")
axs[1].stem(freqs, amplitude_adjust)
axs[1].set_xlabel("Frequency (Hz)")
axs[1].set_ylim(0,.5)

plt.show()

To see the change reflected in the waveform,
we'll have to recombine the amplitude and phase spectra
and transform back from the frequency domain into the time domain.

We need to convert the amptlide and phase information back the Fourier representation.
First, remember that we scaled the amplitude spectrum by the length of the signal -- we need to scale it back.
Secondly, we've been expressing the phase information in degrees, but the Fourier representation is in radians.
Then, we can combine the amplitude and phase information into a complex number.

In [ ]:
# Put spectra in correct units
amplitude_components = amplitude_adjust * len(amplitude_adjust)
phase_components = deg2rad(phase_adjust)

# Put into complex form
F_adjust = amplitude_components * exp(phase_components*1j)

In [ ]:
print(F_adjust)

Now, we can use the _inverse_ DFT to go from the frequency domain back to the signal domain (time).
For this we use the _inverse Fast Fourier Transform_ provided by `ifft`.
This expects the frequencies to be ordered the same as they come out of the FFT:
DC and positive first, then the negatives.
Since we shifted the ordering, we also now have to undo that ordering, for which we can use `ifftshift`.

In [ ]:
# Inverse (Discrete) Fourier Transform (iFFT)
L_adjust = ifft(ifftshift(F_adjust))

Finally, we have our modified signal!

In [ ]:
print(L_adjust.real)

In [ ]:
# Plot
fig, axs = plt.subplots(2,3, figsize=(16, 8), sharey="col", sharex="col")

axs[0,0].set_title("Original waveform")
axs[0,0].plot(times, L)
axs[1,0].set_xlabel("Time (s)")

axs[0,1].set_title("Original amplitude spectrum")
axs[0,1].stem(freqs, amplitude_spectrum)

axs[0,2].set_title("Original phase spectrum")
axs[0,2].stem(freqs, phase_spectrum)

axs[1,1].set_title("Adjusted amplitude")
axs[1,1].stem(freqs, amplitude_adjust)
axs[1,1].set_xlabel("Frequency (Hz)")

axs[1,2].set_title("Adjusted phase spectrum")
axs[1,2].stem(freqs, phase_adjust)
axs[1,2].set_xlabel("Frequency (Hz)")

axs[1,0].set_title("New waveform")
axs[1,0].plot(times, L_adjust.real)

plt.show()

### Adjusting amplitude and phase of single component

We can also do manipulations of a single sinusoid,
even if we have multiple components in our signal.

Let's look at compound waveform, and adjust the amplitude of one component:

In [ ]:
# Original
L_1 = sinusoid(frequency=original_frequency, amplitude=original_amplitude, phase=original_phase)
L_2 = sinusoid(frequency=5*original_frequency, amplitude=2*original_amplitude, phase=original_phase+45)
L_3 = sinusoid(frequency=10*original_frequency, amplitude=.3*original_amplitude, phase=original_phase+60)

L = L_1 + L_2 + L_3 + .25

# Fourier transform
FL = fftshift(fft(L))
FL = FL.round()
FL.real[FL.real == -0] = 0
FL.imag[FL.imag == -0] = 0

# Derive amplitude & phase spectra
freqs = fftshift(fftfreq(len(times), d=(1/sampling_rate)))
amplitude_spectrum = np.abs(FL) / (duration * sampling_rate)
phase_spectrum = np.rad2deg(np.angle(FL))

In [ ]:
# Plot
fig, axs = plt.subplots(1,3, figsize=(16, 4), sharey="col", sharex="col")

axs[0].set_title("Original waveform")
axs[0].plot(times, L)
axs[0].set_xlabel("Time (s)")

axs[1].set_title("Original amplitude spectrum")
axs[1].stem(freqs, amplitude_spectrum)

axs[2].set_title("Original phase spectrum")
axs[2].stem(freqs, phase_spectrum)

plt.show()

In [ ]:
# Which frequency, and what indices is that?
component = 1 # Hz
indices = [np.where(freqs==component)[0][0], np.where(freqs==-component)[0][0]]

# Adjust amplitude spectrum
amplitude_adjust = amplitude_spectrum.copy()
amplitude_adjust[indices] = .9

# Adjust phase spectrum
phase_adjust = phase_spectrum.copy()
phase_adjust[indices] = [-120,120]

In [ ]:
# Plot
fig, axs = plt.subplots(1,3, figsize=(16, 4), sharey="col", sharex="col")

# axs[0,0].set_title("Original waveform")
# axs[0,0].plot(times, L)
# axs[1,0].set_xlabel("Time (s)")

# axs[0,1].set_title("Original amplitude spectrum")
# axs[0,1].stem(freqs, amplitude_spectrum)

# axs[0,2].set_title("Original phase spectrum")
# axs[0,2].stem(freqs, phase_spectrum)

axs[1].set_title("Adjusted amplitude spectrum")
axs[1].stem(freqs, amplitude_adjust)
axs[1].stem(freqs[indices], amplitude_adjust[indices], markerfmt="C1o",linefmt="C1-")
axs[1].set_xlabel("Frequency (Hz)")

axs[2].set_title("Adjusted phase spectrum")
axs[2].stem(freqs, phase_adjust)
axs[2].stem(freqs[indices], phase_adjust[indices], markerfmt="C1o",linefmt="C1-")
axs[2].set_xlabel("Frequency (Hz)")

axs[0].set_title("New waveform")

plt.show()

In [ ]:
amplitude_components = amplitude_adjust * len(amplitude_adjust)
phase_components = deg2rad(phase_adjust)
F_adjust = amplitude_components * exp(phase_components*1j)
L_adjust = ifft(ifftshift(F_adjust))

In [ ]:
# Plot
fig, axs = plt.subplots(2,3, figsize=(16, 8), sharey="col", sharex="col")

axs[0,0].set_title("Original waveform")
axs[0,0].plot(times, L)
axs[1,0].set_xlabel("Time (s)")

axs[0,1].set_title("Original amplitude spectrum")
axs[0,1].stem(freqs, amplitude_spectrum)

axs[0,2].set_title("Original phase spectrum")
axs[0,2].stem(freqs, phase_spectrum)

axs[1,1].set_title("Adjusted amplitude spectrum")
axs[1,1].stem(freqs, amplitude_adjust)
axs[1,1].stem(freqs[indices], amplitude_adjust[indices], markerfmt="C1o",linefmt="C1-")
axs[1,1].set_xlabel("Frequency (Hz)")

axs[1,2].set_title("Adjusted phase spectrum")
axs[1,2].stem(freqs, phase_adjust)
axs[1,2].stem(freqs[indices], phase_adjust[indices], markerfmt="C1o",linefmt="C1-")
axs[1,2].set_xlabel("Frequency (Hz)")

axs[1,0].set_title("New waveform")
axs[1,0].plot(times, L_adjust.real)

plt.show()

We can adjust the amplitude and phase of one of our components,
while keeping the other components the same.

We see that by increasing the amplitude of that component,
it becomes more "prominent" in the waveform,
since a larger share of the total amplitude of the compound signal now comes from that component.

Since we can express the differences in amplitude and phase spectra
between the original and adjusted signal
as a single summation/subtraction,
we can also express the differences between the waveforms as such;
a single sinusoid, with a particular amplitude and phase,
can be added to the original to achieve the adjusted signal.

In [ ]:
fig, (orig, adjust, diff) = plt.subplots(1,3, figsize=(18,5), sharex="all", sharey="row")

plot_waveform(orig, L, times=times)
orig.set_title("Original")
orig.set_xlabel("")

plot_waveform(adjust, L_adjust.real, times=times)
adjust.set_title("Adjusted")
adjust.set_ylabel("")

plot_waveform(diff, L_adjust.real - L, times=times)
diff.set_title("Difference (A - O)")
diff.set_ylabel("")
diff.set_xlabel("")

plt.show()

But imagine that if you want to do this manipulation in the signal domain (time):
then you would have to:
- know what you want the adjusted amplitude and phase to be
- know what the original amplitude and phase of the component are
- subtract the amplitude and phase

to construct the difference signal.
In the frequency domain, we only had to specify the adjusted amplitude and phase.

### Removing components

Another common manipulation that we may do in the frequency domain
is to _remove_ frequency components in some range,
e.g., above some cutoff frequency.
For example, the cell/system that you're studying
may not be responsive above some frequency,
so you want to remove all the higher frequencies from your stimulus.

Let's try this out on our signal where we added random noise
to each timepoint in the waveform
-- in principle, we don't know any of the frequency properties of the noise.

In [ ]:
# Fourier transform
Fy = fftshift(fft(L_in_w_noise))

# Clean
Fy = Fy.round()
Fy.real[Fy.real == -0] = 0
Fy.imag[Fy.imag == -0] = 0

# Derive spectra
amplitude_spectrum = np.abs(Fy) / len(Fy)
phase_spectrum = np.rad2deg(np.angle(Fy))

In [ ]:
# Plot
fig, axs = plt.subplots(1,3, figsize=(16, 4), sharey="col", sharex="col")

axs[0].set_title("Original waveform")
axs[0].plot(times, L_in_w_noise)
axs[0].set_xlabel("Time (s)")

axs[1].set_title("Original amplitude spectrum")
axs[1].stem(freqs, amplitude_spectrum)

axs[2].set_title("Original phase spectrum")
axs[2].stem(freqs, phase_spectrum)

plt.show()

As we saw before,
the frequency domain representation shows that we have a signal,
embedded in the noise.
That is, there is a higher amplitude signal, with a clear, low, frequency.

Also, we see that the noise is characterized by all frequency bands
with varying amplitude
and, crucially, random phase.

Let's remove some of this noise.
Since we know that the signal we're looking for has a low frequency,
we can filter out all high frequency components
and we should be left with a less noisy signal.
In other words, we'll find all frequency components above some cutoff frequency
and set the amplitudes of all these components to 0.
To keep things proper, we'll have to do the same with the negative frequencies,
that is, we want to cut off all components where the absolute frequency is higher than our cutoff.
We don't have to alter the phase of any components,
since the phase of a component with 0 amplitude is irrelevant.

In [ ]:
# What is the cutoff frequency?
cutoff = 5 # Hz

# Adjust amplitude spectrum
amplitude_adjust = amplitude_spectrum.copy()
amplitude_adjust[abs(freqs) > cutoff] = 0

# Adjust phase spectrum
phase_adjust = phase_spectrum.copy()

In [ ]:
# Plot
fig, axs = plt.subplots(2,3, figsize=(16, 8), sharey="col", sharex="col")

axs[0,0].set_title("Original waveform")
axs[0,0].plot(times, L)
axs[1,0].set_xlabel("Time (s)")

axs[0,1].set_title("Original amplitude spectrum")
axs[0,1].stem(freqs, amplitude_spectrum)

axs[0,2].set_title("Original phase spectrum")
axs[0,2].stem(freqs, phase_spectrum)

axs[1,1].set_title("Adjusted amplitude spectrum")
axs[1,1].stem(freqs, amplitude_adjust)
axs[1,1].stem(freqs[freqs > cutoff], amplitude_adjust[freqs > cutoff], markerfmt="C1o",linefmt="C1-")
axs[1,1].stem(freqs[freqs < -cutoff], amplitude_adjust[freqs < -cutoff], markerfmt="C1o",linefmt="C1-")
axs[1,1].set_xlabel("Frequency (Hz)")

axs[1,2].set_title("Adjusted phase spectrum")
axs[1,2].stem(freqs, phase_adjust)
axs[1,2].stem(freqs[freqs > cutoff], phase_adjust[freqs > cutoff], markerfmt="C1o",linefmt="C1-")
axs[1,2].stem(freqs[freqs < -cutoff], phase_adjust[freqs < -cutoff], markerfmt="C1o",linefmt="C1-")
axs[1,2].set_xlabel("Frequency (Hz)")

axs[1,0].set_title("New waveform")

plt.show()

Now that we have altered the amplitude spectrum,
we can construct the adjusted waveform
through the inverse DFT as before:

In [ ]:
# Put spectra in correct units
amplitude_components = amplitude_adjust * len(amplitude_adjust)
phase_components = deg2rad(phase_adjust)

# Put into complex form
F_adjust = amplitude_components * exp(phase_components*1j)

# Inverse DFT
L_adjust = ifft(ifftshift(F_adjust))

In [ ]:
# Plot
fig, axs = plt.subplots(2,3, figsize=(16, 8), sharey="col", sharex="col")

axs[0,0].set_title("Original waveform")
axs[0,0].plot(times, L_in_w_noise)
axs[1,0].set_xlabel("Time (s)")

axs[0,1].set_title("Original amplitude spectrum")
axs[0,1].stem(freqs, amplitude_spectrum)

axs[0,2].set_title("Original phase spectrum")
axs[0,2].stem(freqs, phase_spectrum)

axs[1,1].set_title("Adjusted amplitude spectrum")
axs[1,1].stem(freqs, amplitude_adjust)
axs[1,1].stem(freqs[freqs > cutoff], amplitude_adjust[freqs > cutoff], markerfmt="C1o",linefmt="C1-")
axs[1,1].stem(freqs[freqs < -cutoff], amplitude_adjust[freqs < -cutoff], markerfmt="C1o",linefmt="C1-")
axs[1,1].set_xlabel("Frequency (Hz)")

axs[1,2].set_title("Adjusted phase spectrum")
axs[1,2].stem(freqs, phase_adjust)
axs[1,2].stem(freqs[freqs > cutoff], phase_adjust[freqs > cutoff], markerfmt="C1o",linefmt="C1-")
axs[1,2].stem(freqs[freqs < -cutoff], phase_adjust[freqs < -cutoff], markerfmt="C1o",linefmt="C1-")
axs[1,2].set_xlabel("Frequency (Hz)")

axs[1,0].set_title("New waveform")
axs[1,0].plot(times, L_adjust.real)

plt.show()

This is indeed a lot "cleaner":
since we have remove high frequencies,
there are fewer rapid large changes.

Note that this kind of operation
is much easier to do in the frequency domain than in the signal domain.
Like in the previous example
we could figure out some signal,
which when we subtract it from the original waveform
removes the noise components and leaves us with the "clean" waveform.
However, to do that,
we would first have to characterize the amplitude and phase
of each of the components we want to remove.
Then, we would have to construct a waveform for each of these components
with the right amplitude and phase, and subtract it from the original.


In the frequency domain,
we just specify that everything beyond a certain frequency
should be set to 0.

### Filtering: cutting of components by multiplying amplitude spectrum

Rather than setting all these components to 0 "by hand"
we can use a fun trick: multiplying the amplitude spectrum by 0s and 1s.
For the components that we want to remove,
we multiply the amplitude by 0,
so that the amplitude of that component becomes 0.
The other components we don't want to change,
so we multiply those by 1.

Thus, we need to have a vector of the same length as the amplitude spectrum,
with a 0 for each component we want to cut
and a 1 for each component we want to keep:

In [ ]:
# Adjust spectra:
cutoff = 5

discard_high_freqs = (freqs > cutoff) + (freqs < -cutoff)
keep_freqs = ~discard_high_freqs

In Python we can use an extra fun trick:
that the values `True` and `False` can automatically used as `1` and `0`, resp.
Thus, a vector of `True`s and `False`s is already our vector of `1`s and `0`s.

In [ ]:
print(keep_freqs)

In [ ]:
print(keep_freqs * 1)

In [ ]:
amplitude_adjust = amplitude_spectrum * keep_freqs

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(6, 4), sharex='all', sharey='all')

axs[0].stem(freqs, amplitude_spectrum)
axs[0].set_xlabel("Frequency (Hz)")
axs[0].set_title("Original amplitude spectrum")

axs[1].set_xlabel("Frequency (Hz)")
axs[1].set_title("Filter")
axs[1].step(freqs, keep_freqs)

plt.show()

In [ ]:
fig, axs = plt.subplots(1,1, figsize=(3, 4), sharex='all', sharey='all')

axs.set_xlabel("Frequency (Hz)")
axs.set_title("Filtered amplitude spectrum")
axs.stem(freqs, amplitude_adjust)

plt.show()

This vector of which frequencies to keep, and which to discard is a filter.
Multiplying the amplitude spectrum with this filter
then filters out the components for which the filter is 0,
and lets the components pass through for which the filter is 1.

In [ ]:
def plot_filtered(cutoff_low=0., cutoff_high=15.):
    # Fourier transform
    Fy = fftshift(fft(L_in_w_noise))

    # Clean
    Fy = Fy.round()
    Fy.real[Fy.real == -0] = 0
    Fy.imag[Fy.imag == -0] = 0

    # Derive spectra
    amplitude_spectrum = np.abs(Fy) / len(Fy)
    phase_spectrum = np.rad2deg(np.angle(Fy))
    
    # Design filter:
    discard_idcs = (freqs < cutoff_low) + (freqs > cutoff_high)
    discard_idcs_neg = (freqs > -cutoff_low) + (freqs < -cutoff_high)
    filter_idcs = ~discard_idcs + ~discard_idcs_neg

    # Apply filter
    amplitude_adjust = amplitude_spectrum * filter_idcs
    phase_adjust = phase_spectrum.copy()

    # Put spectra in correct units
    amplitude_components = amplitude_adjust * len(amplitude_adjust)
    phase_components = deg2rad(phase_adjust)

    # Put into complex form
    F_adjust = amplitude_components * exp(phase_components*1j)

    # Inverse DFT
    L_adjust = ifft(ifftshift(F_adjust))

    # Plot
    fig, axs = plt.subplots(2,3, figsize=(16, 8), sharey="col", sharex="col")

    axs[0,0].set_title("Original waveform")
    axs[0,0].plot(times, L_in_w_noise)
    axs[1,0].set_xlabel("Time (s)")

    axs[0,2].set_title("Original amplitude spectrum")
    axs[0,2].stem(freqs, amplitude_spectrum)

    fig.delaxes(axs[0,1])
    fig.delaxes(axs[1,1])

    plt.subplot(3,3,5)
    plt.step(freqs, filter_idcs)
    plt.title("Filter")
    plt.xlabel("Frequency (Hz)")
    plt.ylim(-.1,1.1)

    axs[1,2].set_title("Adjusted amplitude spectrum")
    axs[1,2].stem(freqs, amplitude_adjust)
    axs[1,2].set_xlabel("Frequency (Hz)")


    axs[1,0].set_title("New waveform")
    axs[1,0].plot(times, L_adjust.real)


max_slider = widgets.FloatSlider(min=0, value=(freqs.max()+1/duration)/2, max=freqs.max()+(1/duration))
min_slider = widgets.FloatSlider(min=0, value=0, max=max_slider.value)
def update_max(*args):
    min_slider.max = max_slider.value
    if min_slider.value > min_slider.max:
        min_slider.value = min_slider.max
def update_min(*args):
    max_slider.min = min_slider.value
    if max_slider.value < max_slider.min:
        max_slider.value = max_slider.min
max_slider.observe(update_max)
min_slider.observe(update_min)
    
widgets.interactive(plot_filtered, cutoff_high=max_slider, cutoff_low=min_slider)

- Low-pass filter cuts off high frequencies, and lets low frequencies pass (i.e., `cutoff_low = 0`)
- High-pass filter cuts off low frequencies, and lets high frequencies pass (i.e., `cutoff_high >= max`)
- Bandpass filter cuts off both some low frequencies and some high frequencies, and lets a band of frequencies (between the two cutoffs) pass (i.e., `cutoff_low > 0 & cutoff_high < max`)

Filter does not have to be binary; can take any value and will scale amplitude accordingly.

<h1>Summary<span class="tocSkip"></span></h1>

- Signals: descriptions of change over time
   - Frequency, amplitude, phase, mean
- Frequency-domain description of a signal: Fourier transform
   - Frequency components (inc. DC), amplitude spectrum, phase spectrum
- Signal resolution: important limitations on digital signal processing
   - Sampling rate determines Nyquist limit
   - duration determines fundamental frequency
- Compound waveforms
   - Adding signals adds their amplitude spectra, phase spectra
   - Frequency domain can uncover structure that is not clear in signal domain
- Manipulating signals in the frequency domain 
   - inverse Fourier transform (iFFT)
   - filtering signals, by multiply amplitude spectra